# Add a new feature

Adding a new feature to be computed by the `FeaturesComputer` class is pretty easy and straightforward, and doesn't need to do heavy modification to the core code. The package uses the ability of Python to modify function via decorators to automatically add a function to the list of functions that compute features. 


In [1]:
%cd -q ../
import numpy as np
from wavely.signal.features.features import FeaturesComputer, feature, noexport
from wavely.signal.units.helpers import split_signal

### Simple feature

To add a feature, start writing the function that computes the feature and returns its value. The function can take different values as input:
  * `block`: a numpy array containing the audio data.
  * `n_fft`: the size a the FFT used (only useful for spectrum computation)
  * `window`: a numpy array containing the window that is to be applied to the audio data
  * any other feature registered to be computed by `FeaturesComputer`

Then adding the feature to the class is as simple as adding the `@feature` decorator to the function.

In [2]:
@feature
def example(blocks: np.ndarray, window: np.ndarray) -> float:
    return (blocks * window).std(axis=-1)

### Feature tags and dims
Features are tagged since `v0.4.0`. To add the corresponding tag and dimension to a feature, just specific it in the `@feature` decorator:

In [3]:
@feature("mytag", dims=["blocks"])
def example(blocks: np.ndarray, window: np.ndarray) -> float:
    return (blocks * window).std(axis=-1)

For more information, see the corresponding notebooks `tags.ipynb` and `dims.ipynb`.

#### NOTE:

The feature will be named by the name of your function. It's also important to notice that inputs must have the same exact name of the feature they correspond. For example, adding a feature with a parameter ` bloc` and not `block` will throw an error.

Any array reduction done in the function must be applied to the last axis. This is because audio blocks can either be represented by one dimension arrays in case of a single block computation, or by two dimension arrays for multiple blocks, with blocks data being column-wise.

In [4]:
block_size = 256
rate = 192000
x = np.random.randn(block_size)
blocks = split_signal(x, rate, block_size / rate)

fc = FeaturesComputer(block_size=block_size, rate=rate, n_fft=block_size, window=np.hanning)
fc.compute(blocks)
fc["example"]

/home/maxime/Repositories/signal/wavely/signal/preprocessing/preprocessing.py:701: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


array([0.88622457])

To get the list of every feature that can be use as an input, you can use `fc.allfeatures()`.

In [5]:
fc.allfeatures()

dict_keys(['peak_to_peak_displacement', 'spectralflatness', 'amplitude_envelope', 'bandflatness', 'spectrum', 'temporal_kurtosis', 'spectralcentroid', 'ultrasoundlevel', 'highfrequencycontent', 'crest', 'hilbert_transform', 'binfactor', 'peakfreq', 'audiblelevel', 'normspectrum', 'crest_factor', 'spectralspread', 'example', 'k_factor_acceleration', 'gl_filter', 'gl_acceleration', 'periodogram', 'instantaneous_phase', 'spectralirregularity', 'spectralentropy', 'crest_factor_acceleration', 'spectralskewness', 'bandcrest', 'spectralpower', 'spectralflux', 'rms', 'peak_to_peak', 'velocity', 'melspectrogram', 'instantaneous_frequency', 'band_periodogram', 'spectralkurtosis', 'dft', 'spectralcrest', 'displacement', 'enbw', 'gl_velocity', 'power', 'spectralrolloff', 'spl', 'zero_crossing_rate', 'bandleq'])

### Temporary features

You can also compute temporary features that are not meant to be exported (for example if you want to use a precomputed intermediary value for several features). Simply use the `@noexport` decorator.

In [6]:
@noexport
@feature
def halfwave(blocks: np.ndarray) -> np.ndarray:
    blocks = blocks.copy()
    blocks[blocks < 0.] = 0.
    return blocks

@feature
def example2(halfwave: np.ndarray) -> np.ndarray:
    return halfwave.mean(axis=-1)

In [7]:
fc = FeaturesComputer(block_size=block_size, rate=rate, n_fft=block_size, window=np.hanning)
fc.compute(blocks)
fc["example2"]

array([0.37735171])

### Per object feature

A feature can be added to only one instance of `FeaturesComputer` with the decorator ` @fc.feature `. Other objects won't compute this feature.

In [8]:
fc = FeaturesComputer(block_size=block_size, rate=rate, n_fft=block_size, window=np.hanning)

@fc.feature
def example3(rms: float) -> float:
    return rms**3

fc.compute(blocks)

{'example3': array([0.81641938]), 'spl': array([93.39215545])}

In [9]:
fc = FeaturesComputer(block_size=block_size, rate=rate, n_fft=block_size, window=np.hanning)
fc.compute(blocks)

{'spl': array([93.39215545])}